## 문제 사항
* 가끔 인식 부정확 - save_img/*_difficult.* 참조 - 거리 threshold 늘려서 다소 개선, 해결은 안됨
* 가운데 구획 필요할 수 있음 - 대응 전략

* 오른 쪽 view 자꾸 겨드랑이로 - 자세 약간 수정함, 너무 돌지 않도록 수정 v
* 툴 형상 변경 - 깊이 5cm 정도만 줄이자 에 따라 데이터 다시 수집 필요 v
* 작업 너비 너무 좁음 - 더 확실한 영역 데이터셋 준비 필요 v
* 동작 비효율적 - 일단 sweep은 task space 경로로 v
* 반대편 이동 정상 동작 안됨 - 해결 v
* 이동 후 오프셋에 따라 동작 불가능해지는 상황 발생 - 미묘한 싱귤러리티 등이 원인 - 오프셋에 따른 보정 전략 도입 v
* 깊이 방향도 데이터 기반으로 실제 섹션 반영 / 랜덤노이즈 테스트 v
* 3 칸 이상 나누기 v
* multiprocess lock 문제 ?


In [1]:
import os
import sys
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

Current PC IP: 192.168.0.125
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10


## Set dimensions

In [2]:
# Table dimension
T_Width = 1.8
T_Height = 0.785
T_Depth = 0.734
TABLE_DIMS = np.array((T_Depth,T_Width,T_Height))

# Tool dimensions
TOOL_DIM = [0.32, 0.08]
TOOL_OFFSET = -0.04
ROBOT_Z_ANGLE = np.pi
MARGIN = 0
TRACK_THICKNESS = 0.001

CHAIR_NAME = "chair0"
CHAIR_DIM = (0.37,0.37,0.60)

HOLD_NAME = "hold0"
HOLD_XYZ = (0.445 + 0.25, 0, CHAIR_DIM[2]/2)
HOLD_RPY = (0, -np.pi/2, -np.pi/2)

INDY_BASE_OFFSET = (0.172,0,0.439)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

## Prepare robot and pipeline setting

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


In [4]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [5]:
# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, point=(-gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), 
                     rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2), rpy=(0,0,0))

# chair holder
gscene.create_safe(gtype=GEOTYPE.SPHERE, name=HOLD_NAME, link_name=HOLD_LINK,
                   dims=(0.01,) * 3, center=HOLD_XYZ, rpy=HOLD_RPY, color=(1, 0, 0, 0.5), display=True,
                   collision=False,
                   fixed=True)
holder = pscene.create_binder(bname=HOLD_NAME, gname=HOLD_NAME, _type=Gripper2Tool, point=(0, 0, 0), rpy=(0, 0, 0))
holder.redundancy = {}


# brush
brush_face = pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), rpy=(0,np.pi/2*1,0))

   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
indy = crob.robot_dict[ROBOT_NAME]
# if CONNECT_INDY:
#     with indy:
#         indy.reset_robot()

## move indy to viewing pose

In [7]:
VIEW_POSE = np.deg2rad([  0., -28.,  85.,  -0.,  57., -180])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
gscene.show_pose(VIEW_POSE_EXT)

## Attach to detection server

In [8]:
if ENABLE_DETECT:
    attacth_to_server()

## Get image & set env
* environment: rectangular space with desk on left behind side
* mark1: on any point of right side wall
* mark2: on the corner between right & front side wall
* mark3: on any point of left side wall
* mark4: on right-front corner of the desk
* chair: on center top of the chair
* desk: on right-near top corner

In [9]:
if CONNECT_CAM:
#     rdict = send_recv_demo_cam({1:1}, host=CAM_HOST)
#     rdict = stream_capture_image(ImageType.FirstView, host=CAM_HOST)
    cam_intrins, d_scale = [rdict[key] for key in ["intrins", "depth_scale"]]
else:
    cam_intrins = [1280, 720, 899.05322265625,  899.21044921875, 654.8836669921875, 352.9295654296875]
    d_scale = 0.0002500000118743628
cameraMatrix = [[cam_intrins[2], 0, cam_intrins[4]], [0, cam_intrins[3], cam_intrins[5]], [0,0,1]]
distCoeffs = [0]*5
    # Set color, depth image path
    
if CONNECT_CAM:
    color_img_path = SAVE_DIR + '/color.jpg'
    depth_img_path = SAVE_DIR + '/depth.png'
else:
    color_img_path = DATASET_CAM_DIR + '/color.jpg'
    depth_img_path = DATASET_CAM_DIR + '/depth.png'

# Read color, depth image file, keep 16bit information
color_img_read = cv2.imread(color_img_path, flags=cv2.IMREAD_UNCHANGED)
depth_img_read = cv2.imread(depth_img_path, flags=cv2.IMREAD_UNCHANGED)

Tmc = viewpoint.get_tf(list2dict(VIEW_POSE_EXT, gscene.joint_names), from_link=MOBILE_BASE)
Tcm = SE3_inv(Tmc)
if DETECT_MARKER:
    if CONNECT_CAM:
        Tco_dict, corner_dict = aruco_map.get_object_pose_dict(color_img_read, cameraMatrix, distCoeffs)
    else:
        Tco_dict = {}
        Tco_dict["mark1"] = np.matmul(Tcm, SE3(Rot_rpy((np.pi/2, 0, -np.pi/2)), [2,-2.5,0.3]))
        Tco_dict["mark2"] = np.matmul(Tcm, SE3(Rot_rpy((np.pi/2, 0, -np.pi/2)), [5,-2.6,0.3]))
        Tco_dict["mark3"] = np.matmul(Tcm, SE3(Rot_rpy((np.pi/2, 0, -np.pi/2)), [2,2.5,0.3]))
        Tco_dict["mark4"] = np.matmul(Tcm, SE3(Rot_rpy((np.pi/2, 0, -np.pi/2)), [0.5,1.5,0.3]))
        Tco_dict["table"] = np.matmul(Tcm, SE3(Rot_rpy((np.pi/2, 0, -np.pi/2)), [2,-0.5,0.7]))
        Tco_dict["chair"] = np.matmul(Tcm, SE3(Rot_rpy((np.pi/2, 0, -np.pi/2)), [2,-1,0.6]))
Tmo_dict = {k: np.matmul(Tmc, Tco) for k, Tco in Tco_dict.items()}

for k, Tmo in Tmo_dict.items():
    gscene.create_safe(GEOTYPE.BOX, k+"_m", "base_link", dims=[0.18, 0.18, 0.01],
                       center = Tmo[:3,3], rpy= Rot2rpy(Tmo[:3,:3]),
                       color=(1,0.0,0.0,0.3), display=True, fixed=True, collision=False)
    
m1 = Tmo_dict["mark1"][:2,3]
m2 = Tmo_dict["mark2"][:2,3]
dmark = m2-m1
view_theta = -np.arctan2(dmark[1], dmark[0])
Tbm = SE3(Rot_axis(3, view_theta), [0]*3)

Tbo_dict = {k: np.matmul(Tbm, Tmo) for k, Tmo in Tmo_dict.items()}
for k, Tbo in Tbo_dict.items():
    gscene.create_safe(GEOTYPE.BOX, k+"_m", "base_link", dims=[0.18, 0.18, 0.01],
                       center = Tbo[:3,3], rpy= Rot2rpy(Tbo[:3,:3]),
                       color=(1,0.0,0.0,0.3), display=True, fixed=True, collision=False)
    
VIEW_LOC[2] = view_theta
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
gscene.show_pose(VIEW_POSE_EXT)
_ =viewpoint.draw_traj_coords([VIEW_POSE_EXT])

x_max = Tbo_dict["mark2"][0,3]
x_min = -1
y_max = Tbo_dict["mark3"][1,3]
y_min = Tbo_dict["mark1"][1,3]
gscene.set_workspace_boundary( x_min, x_max, y_min, y_max, -CLEARANCE, 3, thickness=WALL_THICKNESS)


xy4 = Tbo_dict["mark4"][:2,3]
desk_height = 1.0
gscene.create_safe(GEOTYPE.BOX, "work_desk", "base_link", dims=[xy4[0]-x_min, y_max-xy4[1], desk_height],
                   center = ((x_min+xy4[0])/2, (y_max+xy4[1])/2, desk_height/2), rpy= (0,0,0),
                   color=(0.8,0.4,0.0,0.1), display=True, fixed=True, collision=True)


Tbt = np.matmul(Tbo_dict["table"], SE3(Rot_rpy((np.pi/2, 0, np.pi)), [-T_Depth/2, 0, -T_Width/2]))
gscene.create_safe(GEOTYPE.BOX, "table", "base_link", dims=TABLE_DIMS,
                   center = Tbt[:2,3].tolist()+[T_Height/2], rpy= Rot2rpy(Tbt[:3,:3]),
                   color=(0.8,0.8,0.0,1), display=True, fixed=True, collision=True)


Tbc_m = Tbo_dict["chair"]
gscene.create_safe(GEOTYPE.CYLINDER, CHAIR_NAME, "base_link", dims=CHAIR_DIM,
                   center = Tbc_m[:2,3].tolist()+[CHAIR_DIM[2]/2], rpy= (0,0,0),
                   color=(0.8,0.8,0.0,1), display=True, fixed=False, collision=True)

chair_bound = gscene.create_safe(GEOTYPE.CYLINDER, CHAIR_NAME+"put_ban", "base_link", dims=np.add(CHAIR_DIM, [0.4,0.4,0]),
                                 center = (0,0,0), rpy= (0,0,0),
                                 color=(0.8,0.8,0.0,0.1), display=True, fixed=False, collision=False, parent=CHAIR_NAME)

### In case of table detection

In [10]:
# Output of inference(mask for detected table)
if ENABLE_DETECT:
    if not DETECT_MARKER_ONLY:
        mask_out = detect_from_server(color_img_read)
        ICP_result1 = None
        if mask_out is not None:
            plt.imshow(mask_out)
            # Crop masking part
            vis_mask = (mask_out * 255).astype('uint8')
            color_instance = cv2.bitwise_and(color_img_read, color_img_read, mask=vis_mask).astype(np.uint16)
            depth_instance = cv2.bitwise_and(depth_img_read, depth_img_read, mask=vis_mask).astype(np.uint16)
            cv2.imwrite(CROP_DIR + '/color_crop.jpg', color_instance)
            cv2.imwrite(CROP_DIR + '/depth_crop.png', depth_instance)

            set_cam_params(cam_intrins, d_scale)
            model_mesh, pcd_out = preprocessing()
            ICP_result1 = compute_ICP(model_mesh, pcd_out, (-T_Height, -T_Depth, 0.0))
            #ICP_result2 = compute_ICP(model_mesh, pcd2)

        if ICP_result1 is None: # test
            ICP_result1 = np.array([[ 0.97952723,  0.00354742,  0.20128047, -0.63449415],
                                    [ 0.08693341,  0.89435887, -0.43882204, -0.18267728],
                                    [-0.18157366,  0.44733614,  0.87574048,  1.77040063],
                                    [ 0.        ,  0.        ,  0.        ,  1.        ]])
    OFF_DIR = np.array([1,1,-1])
    # OFF_DIR = np.array([1,1,0])
    #TABLE_DIMS[[0,1,2]]
    #np.hstack([TABLE_DIMS[[0,1]], [0]])
    T_toff = SE3(Rot_axis(3,np.pi), np.divide(TABLE_DIMS[[0,1,2]]*OFF_DIR,2))
    T_co = np.matmul(np.matmul(ICP_result1, SE3(Rot_axis(1, np.pi/2), [0]*3)), T_toff)
    T_lo = np.matmul(viewpoint.Toff, T_co)
    T_bo=np.matmul(T_bc, T_co)

    # fit to plane
    floor_g = gscene.NAME_DICT["floor"]
    floor_height = floor_g.center[2] + floor_g.dims[2]/2

    # Floor Fitting
    T_bo[2,3] =  floor_height+TABLE_DIMS[2]/2

    azimuth, zenith = mat2hori(T_bo[:3,:3])
    zenith_up = np.pi-zenith
    Raz = Rot_axis(3, azimuth)
    Rzn = Rot_axis(2, -zenith_up)
    Roff = np.matmul(np.matmul(Raz, Rzn), Raz.transpose())
    T_bo[:3,:3] = np.matmul(Roff, T_bo[:3,:3])

    gscene.add_highlight_axis("table", "center", link_name="base_link", center=T_bo[:3,3], orientation_mat=T_bo[:3,:3])

    # geometry 추가
    table_prev = add_table(gscene, "table", TABLE_DIMS, T_bo, collision=False)

## select task area

In [11]:
TABLE_HEIGHT = TABLE_DIMS[2]
T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_DEPTH_OFF = T_brush_e[0, 3]
EE_HEIGHT = TABLE_HEIGHT + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]
sweep_width, (area_width, area_depth), width_range, divisions, div_num, area_center = select_task_area(
    robot_config, TABLE_DIMS, TOOL_DIM, EE_DEPTH_OFF, EE_HEIGHT, ROBOT_Z_ANGLE, 
    MARGIN=MARGIN, TOOL_DEPTH_MIN=0.6, TOOL_DEPTH_MAX=1.0)

reference height: 0.531499995947
== MOTION_DEPTH: 0.414 ==
== WIIPE_DEPTH: 0.414 ==
== MOTION_DEPTH: 0.414 ==
== WIIPE_DEPTH: 0.207 ==
sweep depths: (0.48, 0.7)
divisions: (2, 2)


### add waypoints & chair

In [12]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

# floor
floor = pscene.create_binder(bname="floor", gname="floor_ws", _type=PlacePlane, point=None)

# waypoints
TABLE_DIM_DW = TABLE_DIMS[:2]
track_size_dw = TABLE_DIM_DW / (divisions[1], divisions[0])
REF_CENTER_00 = (TABLE_DIM_DW/2 - track_size_dw / 2) * (-1,1)

Trt = SE3(Rot_axis(3,np.pi), tuple(np.negative(area_center))+(TABLE_DIMS[2]-TRACK_THICKNESS/2,)) # offset from indy base to selected track
Tmr = get_tf(to_link=ROBOT_BASE, joint_dict=HOME_DICT, urdf_content=gscene.urdf_content, from_link=MOBILE_BASE) # from mobile to indy
Ttm = SE3_inv(np.matmul(Tmr, Trt)) # from track to mobile
Tfb = SE3_inv(gscene.NAME_DICT["floor_ws"].get_tf(HOME_DICT)) # from floor to base_link
Tfm_home = np.matmul(Tfb, get_tf(to_link=MOBILE_BASE, joint_dict=HOME_DICT, urdf_content=gscene.urdf_content)) # from floor to mobile home

T_tab_trac_list = []
wp_list = []
for i_d in range(divisions[1]):
    for i_w in range(divisions[0]):
        xyz = (tuple((REF_CENTER_00 + track_size_dw * (i_d, -i_w)) * ((1, -1) if i_d == 1 else (1, 1)))
               +(TABLE_DIMS[2]/2-TRACK_THICKNESS/2,))
        rpy = (0,0,np.pi) if i_d == 1 else (0,0,0)
        track_tmp = gscene.create_safe(GEOTYPE.BOX, "track_{}_{}".format(i_d, i_w), "base_link",
                                       dims = tuple(track_size_dw)+(TRACK_THICKNESS,), center = xyz, rpy = rpy, 
                                       color=(0.0,0.0,0.8,0.3), display=True, fixed=True, collision=False, 
                                       parent="table")
        Tft = np.matmul(Tfb, track_tmp.get_tf(HOME_DICT))
        Tfm = np.matmul(Tft, Ttm)
        xyz, rpy = T2xyzrpy(Tfm)
        wp_list.append(
            gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(len(wp_list)), link_name="base_link",
                               dims=WP_DIMS, center=tuple(xyz[:2])+(0,), 
                               rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, 
                               parent="floor_ws")
        )
        
# add home waypoint
xyz, rpy = T2xyzrpy(Tfm_home)
wp_list.append(
    gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(len(wp_list)), link_name="base_link",
                       dims=WP_DIMS, center=tuple(xyz[:2])+(0,), 
                       rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, parent="floor_ws")
)
wp_task = pscene.create_subject(oname="waypoints", gname="floor_ws", _type=WayopintTask, 
                                action_points_dict={wp.name: WayFrame(wp.name, wp, [0,0,WALL_THICKNESS/2], [0,0,0]) 
                                                    for wp in wp_list})

# chair
chair = pscene.create_subject(oname=CHAIR_NAME, gname=CHAIR_NAME, _type=CylinderObject, 
                              GRASP_WIDTH_MIN=CHAIR_DIM[0]-0.1, GRASP_WIDTH_MAX=CHAIR_DIM[0]+0.1, 
                              GRASP_DEPTH_MIN=CHAIR_DIM[0]/2, GRASP_DEPTH_MAX=CHAIR_DIM[0]/2)
chair.action_points_dict[CHAIR_NAME+"_side_g"].redundancy = {'u': (-np.pi, np.pi)}

##  Set planner

In [13]:
from pkg.planning.task.rrt_star import TaskRRTstar
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler


tplan = TaskRRTstar(pscene)
tplan.REWIND_MAX = 3
tplan.new_node_sampler = PenaltyNodeSampler(3, 3)
tplan.parent_node_sampler = UniformNodeSampler(3)
tplan.explicit_edges = {
    ("hold0", i_w): [("floor_ws", i_w)] for i_w in range(len(wp_list)+1)}
# Q_indy_hold_ref = [ 1.30471890e-01,  1.21258767e+00, 4.29336051e-01, -1.58105239e+00,  1.43938764e+00,  7.18456766e-02]
# HOLD_POSE_EXT = np.array(VIEW_LOC + Q_indy_hold_ref)
initial_state = pscene.initialize_state(VIEW_POSE_EXT)
goal_nodes = [('floor_ws',)+(len(wp_list)-1,)]

## Plan

In [14]:
pscene.set_object_state(initial_state)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

mplan.reset_log(False)
mplan.motion_filters[0].put_banned = [chair_bound]

ppline.search(initial_state, goal_nodes, verbose=True,
              timeout=0.3, timeout_loop=5, multiprocess=True,
              add_homing=False, terminate_on_first=False, post_optimize=False)
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule_mobile = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
# ppline.play_schedule(snode_schedule_mobile)

Use 20/20 agents
try: 0 - ('floor_ws', 0)->('floor_ws', 1)
try: 0 - ('floor_ws', 0)->('hold0', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 0 - ('floor_ws', 0)->('floor_ws', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 0)->('hold0', 0) = fail
try: 0 - ('floor_ws', 0)->('hold0', 0)
Motion Filer Failure: GraspChecker
try transition motion
try: 0 - ('floor_ws', 0)->('floor_ws', 1)
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try: 0 - ('floor_ws', 0)->('hold0', 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 0)->('hold0', 0) = fail
try: 0 - ('floor_ws', 0)->('hold0', 0)
transition motion tried: True
result: 0 - ('floor_ws', 0)->('floor_ws', 1) = fail
try transition motion
try: 0 - ('floor_ws', 0)->('hold0', 0)
result: 0 - ('floor_ws', 0)->('hold0', 0) = success
try transition motion
branching: 0->1 (0.18/5.0 s, steps/err: 25(94.8610305786 ms)/0.00176981401462)
try: 1

try transition motion
try: 9 - ('hold0', 0)->('floor_ws', 0)
branching: 0->9 (0.78/5.0 s, steps/err: 63(244.777202606 ms)/0.000886176201976)
try transition motion
branching: 5->10 (0.79/5.0 s, steps/err: 28(234.443902969 ms)/0.00199920173969)
try transition motion
try: 9 - ('hold0', 0)->('floor_ws', 0)
transition motion tried: True
try transition motion
transition motion tried: True
Motion Filer Failure: GraspChecker
transition motion tried: True
transition motion tried: True
Motion Filer Failure: GraspChecker
result: 9 - ('hold0', 0)->('floor_ws', 0) = fail
try: 9 - ('hold0', 0)->('floor_ws', 0)
try: 11 - ('floor_ws', 0)->('floor_ws', 1)
try: 4 - ('floor_ws', 0)->('floor_ws', 1)
transition motion tried: True
transition motion tried: True
reserve 11 -> 10
try transition motion
transition motion tried: True
reserve 14 -> 10
try transition motion
result: 5 - ('floor_ws', 0)->('hold0', 0) = success
result: 4 - ('floor_ws', 0)->('floor_ws', 1) = success
branching: 5->12 (0.89/5.0 s, steps/

transition motion tried: True
try: 26 - ('floor_ws', 1)->('floor_ws', 2)
try transition motion
result: 9 - ('hold0', 0)->('floor_ws', 0) = success
result: 14 - ('floor_ws', 0)->('floor_ws', 1) = success
result: 13 - ('floor_ws', 0)->('hold0', 0) = success
branching: 9->25 (1.17/5.0 s, steps/err: 71(334.758043289 ms)/0.00165303020173)
try transition motion
transition motion tried: True
try transition motion
try transition motion
branching: 14->30 (1.17/5.0 s, steps/err: 29(205.579042435 ms)/0.000607871284003)
result: 8 - ('hold0', 0)->('floor_ws', 0) = success
try transition motion
reserve 28 -> 20
branching: 13->31 (1.17/5.0 s, steps/err: 51(136.940002441 ms)/0.000545043098921)
transition motion tried: True
branching: 8->29 (1.18/5.0 s, steps/err: 42(316.184997559 ms)/0.00184523692972)
result: 11 - ('floor_ws', 0)->('floor_ws', 1) = success
result: 14 - ('floor_ws', 0)->('hold0', 0) = success
result: 21 - ('floor_ws', 1)->('floor_ws', 2) = success
transition motion tried: True
try: 25 

try: 49 - ('floor_ws', 1)->('floor_ws', 2)
try transition motion
reserve 41 -> 34
reserve 41 -> 15
reserve 44 -> 24
reserve 41 -> 18
reserve 44 -> 30
transition motion tried: True
reserve 48 -> 46
try transition motion
reserve 41 -> 20
try: 47 - ('floor_ws', 1)->('floor_ws', 2)
reserve 41 -> 24
try transition motion
result: 25 - ('floor_ws', 0)->('floor_ws', 1) = success
result: 37 - ('floor_ws', 1)->('floor_ws', 2) = success
transition motion tried: True
try: 49 - ('floor_ws', 1)->('floor_ws', 2)
result: 19 - ('floor_ws', 0)->('hold0', 0) = success
reserve 44 -> 32
result: 43 - ('floor_ws', 1)->('floor_ws', 2) = success
result: 40 - ('floor_ws', 1)->('floor_ws', 2) = success
reserve 41 -> 30
branching: 37->53 (1.45/5.0 s, steps/err: 9(173.413038254 ms)/0.000655033343874)
branching: 19->52 (1.46/5.0 s, steps/err: 53(294.356107712 ms)/0.0014629486509)
branching: 43->54 (1.46/5.0 s, steps/err: 9(63.7130737305 ms)/0.000824878506038)
branching: 40->50 (1.46/5.0 s, steps/err: 9(110.40902137

reserve 68 -> 64
try transition motion
try transition motion
reserve 69 -> 55
branching: 47->68 (1.67/5.0 s, steps/err: 9(143.087148666 ms)/0.000745074517279)
try transition motion
reserve 69 -> 35
transition motion tried: True
reserve 70 -> 65
transition motion tried: True
try: 67 - ('floor_ws', 2)->('floor_ws', 3)
transition motion tried: True
try transition motion
reserve 69 -> 36
transition motion tried: True
reserve 69 -> 46
reserve 69 -> 50
reserve 73 -> 65
reserve 72 -> 55
reserve 70 -> 64
try: 66 - ('floor_ws', 2)->('floor_ws', 3)
reserve 69 -> 54
try: 66 - ('floor_ws', 2)->('hold0', 2)
reserve 71 -> 55
result: 48 - ('floor_ws', 1)->('floor_ws', 2) = success
transition motion tried: True
reserve 73 -> 64
reserve 71 -> 35
result: 47 - ('floor_ws', 1)->('floor_ws', 2) = success
reserve 74 -> 65
branching: 48->70 (1.73/5.0 s, steps/err: 9(161.30399704 ms)/0.00143653461553)
try transition motion
reserve 72 -> 35
reserve 72 -> 36
branching: 47->73 (1.74/5.0 s, steps/err: 9(233.81400

...

try: 169 - ('floor_ws', 3)->('floor_ws', 4)
reserve 167 -> 56
transition motion tried: True
reserve 167 -> 58
transition motion tried: True
reserve 167 -> 80
transition motion tried: True
reserve 167 -> 15
transition motion tried: True
reserve 167 -> 18
try: 77 - ('floor_ws', 2)->('hold0', 2)
try transition motion
result: 25 - ('floor_ws', 0)->('hold0', 0) = success
transition motion tried: False
branching: 25->171 (4.04/5.0 s, steps/err: 67(352.44011879 ms)/0.00208971949505)
Motion Plan Failure
reserve 167 -> 20
Motion Filer Failure: GraspChecker
result: 79 - ('floor_ws', 2)->('hold0', 2) = success
result: 156 - ('hold0', 3)->('floor_ws', 3) = success
try: 172 - ('floor_ws', 3)->('floor_ws', 4)
branching: 79->170 (4.05/5.0 s, steps/err: 94(349.287986755 ms)/0.00120110438747)
result: 77 - ('floor_ws', 2)->('hold0', 2) = fail
try: 61 - ('floor_ws', 1)->('hold0', 1)
branching: 156->172 (4.06/5.0 s, steps/err: 43(248.845100403 ms)/0.00149959098641)
reserve 173 -> 147
result: 23 - ('floor_

Motion Plan Failure
result: 82 - ('floor_ws', 2)->('hold0', 2) = fail
reserve 185 -> 135
transition motion tried: True
reserve 184 -> 14
try: 72 - ('floor_ws', 1)->('hold0', 1)
reserve 184 -> 19
reserve 185 -> 136
Motion Filer Failure: GraspChecker
reserve 184 -> 25
try transition motion
reserve 184 -> 28
try: 60 - ('floor_ws', 2)->('hold0', 2)
result: 166 - ('floor_ws', 3)->('floor_ws', 4) = fail
reserve 184 -> 29
reserve 185 -> 142
transition motion tried: True
try transition motion
result: 173 - ('hold0', 3)->('floor_ws', 3) = success
result: 121 - ('hold0', 2)->('floor_ws', 2) = fail
reserve 185 -> 144
branching: 173->186 (4.39/5.0 s, steps/err: 14(187.659025192 ms)/0.00124392392377)
try transition motion
reserve 184 -> 41
Motion Filer Failure: GraspChecker
try: 109 - ('hold0', 2)->('floor_ws', 2)
try: 186 - ('floor_ws', 3)->('floor_ws', 4)
reserve 185 -> 146
reserve 184 -> 44
try: 187 - ('floor_ws', 3)->('floor_ws', 4)
result: 174 - ('hold0', 3)->('floor_ws', 3) = success
reserve 

reserve 196 -> 177
transition motion tried: True
transition motion tried: True
reserve 196 -> 179
transition motion tried: False
reserve 200 -> 165
reserve 196 -> 186
try transition motion
reserve 200 -> 166
try transition motion
reserve 203 -> 65
Goal reached
try transition motion
Goal reached
reserve 200 -> 169
reserve 202 -> 55
reserve 200 -> 172
reserve 190 -> 30
reserve 189 -> 60
try: 199 - ('floor_ws', 0)->('floor_ws', 1)
reserve 196 -> 187
result: 187 - ('floor_ws', 3)->('floor_ws', 4) = success
transition motion tried: True
reserve 203 -> 85
Motion Plan Failure
reserve 202 -> 132
result: 186 - ('floor_ws', 3)->('floor_ws', 4) = success
result: 92 - ('hold0', 2)->('floor_ws', 2) = success
reserve 189 -> 76
transition motion tried: False
branching: 92->196 (4.76/5.0 s, steps/err: 56(291.740894318 ms)/0.00123423298792)
try transition motion
reserve 200 -> 176
reserve 205 -> 147
reserve 190 -> 32
reserve 189 -> 86
branching: 186->204 (4.77/5.0 s, steps/err: 44(315.21487236 ms)/0.00

reserve 207 -> 177
reserve 207 -> 179
reserve 208 -> 187
======================= terminated 0: max iteration time reached (1630703273/1630703268.39 s) ===============================
result: 99 - ('floor_ws', 3)->('hold0', 3) = fail
reserve 207 -> 186
reserve 214 -> 160
======================= terminated 8: max iteration time reached (1630703273/1630703268.39 s) ===============================
reserve 202 -> 86
result: 196 - ('floor_ws', 2)->('floor_ws', 3) = success
reserve 206 -> 49
reserve 214 -> 165
branching: 196->216 (5.07/5.0 s, steps/err: 49(259.942054749 ms)/0.00118100342677)
transition motion tried: True
reserve 215 -> 196
reserve 202 -> 196
======================= terminated 12: max iteration time reached (1630703273/1630703268.39 s) ===============================
reserve 206 -> 57
reserve 208 -> 64
reserve 214 -> 166
reserve 218 -> 65
reserve 207 -> 187
transition motion tried: True
reserve 206 -> 69
reserve 208 -> 84
reserve 215 -> 200
reserve 207 -> 64
reserve 202 -> 200

reserve 207 -> 131
reserve 218 -> 177
reserve 208 -> 131
reserve 203 -> 151
reserve 214 -> 88
result: 196 - ('floor_ws', 2)->('floor_ws', 3) = success
reserve 218 -> 179
reserve 218 -> 186
reserve 203 -> 157
branching: 196->223 (5.2/5.0 s, steps/err: 47(385.015964508 ms)/0.00173866874149)
reserve 208 -> 138
result: 196 - ('floor_ws', 2)->('floor_ws', 3) = success
reserve 214 -> 91
transition motion tried: True
reserve 207 -> 138
branching: 196->222 (5.21/5.0 s, steps/err: 48(369.495868683 ms)/0.00140124267877)
reserve 208 -> 149
reserve 207 -> 149
reserve 214 -> 92
reserve 214 -> 93
result: 200 - ('floor_ws', 2)->('floor_ws', 3) = success
reserve 218 -> 187
reserve 203 -> 170
reserve 208 -> 151
======================= terminated 11: max iteration time reached (1630703273/1630703268.39 s) ===============================
reserve 214 -> 98
reserve 208 -> 157
result: 185 - ('floor_ws', 1)->('floor_ws', 2) = success
reserve 214 -> 102
reserve 208 -> 170
branching: 200->224 (5.22/5.0 s, step

## Run

In [15]:
from pkg.planning.motion.moveit.moveit_py import PlannerConfig
state_cur = None
node_cur = None
# T_hold_ref = SE3(Rot_rpy((np.pi/2,0,-np.pi*11/12)), [-0.82,0.12,0.15])
T_hold_ref = SE3(Rot_rpy((np.pi,-0.278333*np.pi,np.pi/2)), [-0.72,-0.10,0.20])
for i_sn, snode in enumerate(snode_schedule_mobile):
    node_pre = node_cur
    state_pre = state_cur
    state_cur= snode.state
    node_cur = snode.state.node
    print("{} -> {}".format(node_pre, node_cur))
    if node_pre is None:
        continue
    ntem_diff = [ntem_cur for ntem_pre, ntem_cur in zip(node_pre, node_cur) if ntem_pre != ntem_cur]
    assert len(ntem_diff) == 1 or i_sn==len(snode_schedule_mobile)-1, "unexpected node change {} -> {}".format(node_pre, node_cur)
    pscene.set_object_state(state_pre)
#     mplan.planner.set_tolerance(0.01, 0.01, 0.001, 0.001, 0.01)
    N_retry = 3
    for i_retry in range(N_retry):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(state_pre, state_cur, 
                                                                               redundancy_dict=snode.redundancy_dict,
                                                                               timeout=2+i_retry*2,
                                                                               plannerconfig=PlannerConfig.RRTstarkConfigDefault
                                                                         )
        if not success:
            print("optimal replanning fail")
            traj_modi = snode.traj.copy()
            traj_modi[:,6:] = state_pre.Q[6:]
            if mplan.validate_trajectory(traj_modi):
                success = True
                Traj, LastQ = traj_modi, traj_modi[-1]
            elif i_retry >= N_retry-1:
                Traj, LastQ, error, success, binding_list = mplan.plan_transition(
                    state_pre, state_cur, redundancy_dict=snode.redundancy_dict, timeout=5,)
                if not success:
                    raise(RuntimeError("plan-time trajectory invalid in runtime"))
        if success:
            snode.traj = Traj
            snode.state.Q = LastQ
            break;
    if CONNECT_MOBILE:
        ## move mobile command
        pass
    else:
        ppline.play_schedule(snode_schedule_mobile[i_sn-1:i_sn+1])
    pscene.set_object_state(state_cur)
        
    if len(ntem_diff) == 1:
        ntem_diff = ntem_diff[0]
        if ntem_diff == HOLD_NAME:
            print("hold motion")
            mplan.update_gscene()
            traj_hold, success = mplan.planner.plan_py(ROBOT_NAME, TIP_LINK,
                                                  np.concatenate(T2xyzquat(T_hold_ref)), ROBOT_BASE, state_cur.Q)
            assert success, "holding fail"
            snode.traj = np.array(list(snode.traj)+list(traj_hold))
            snode.state.Q = traj_hold[-1]
            if CONNECT_INDY:
                ## move robot command
                pass
            else:
                gscene.show_motion(traj_hold)
        elif ntem_diff == "floor_ws":
            print("release motion")
            mplan.update_gscene()
            traj_release, success = mplan.planner.plan_joint_motion_py(ROBOT_NAME,VIEW_POSE, state_cur.Q)
            if not success:
                traj_release=np.array(list(reversed(traj_hold)))
                traj_release[:,:6] = state_cur.Q[:6]
                success = mplan.validate_trajectory(traj_release)
            assert success, "release fail"
            snode.traj = np.array(list(snode.traj)+list(traj_release))
            snode.state.Q = traj_release[-1]
            if CONNECT_INDY:
                ## move robot command
                pass
            else:
                gscene.show_motion(traj_release)
        elif isinstance(ntem_diff, int):
            print("wipe section {}".format(ntem_diff))
            
            pose_cur = snode.state.Q
            track = gscene.NAME_DICT["track_0_0"]
            Tbt = track.get_tf(pose_cur)
            Tbm = get_tf(MOBILE_BASE, list2dict(pose_cur, gscene.joint_names), gscene.urdf_content)
            Ttm = np.matmul(SE3_inv(Tbt), Tbm)

            sweep_list, track_list = make_work_plane(pscene, track, TOOL_DIM, Rtw_ref=Ttm[:3,:3])

            wipe_initial = pscene.initialize_state(pose_cur)
            from_state = wipe_initial.copy(pscene)
            from_state.Q[6:] = 0 # set zero pose to remove planning bias

            ppline.set_task_planner(tplan)
            ppline.set_motion_planner(mplan)
            tplan.explicit_edges = {(0,0):set([(1,0)])}
            ppline.search(from_state, [(2,)*len(wipe_initial.node)], verbose=True,
                          timeout=0.5, timeout_constrained=2, timeout_loop=20, multiprocess=False,
                          add_homing=True, terminate_on_first=True, post_optimize=False, home_pose=pose_cur)
            schedules = ppline.tplan.find_schedules(False)
            schedules_sorted = ppline.tplan.sort_schedule(schedules)
            snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

            # fix starting pose
            snode_start = snode_schedule[1]
            pscene.set_object_state(snode_schedule[0].state)
            trajectory, success = mplan.planner.plan_joint_motion_py(
                ROBOT_NAME, tuple(snode_start.traj[-1, crob.idx_dict[ROBOT_NAME]]), 
                tuple(wipe_initial.Q), timeout=1)
            if success:
                snode_start.traj = trajectory
                
            if CONNECT_INDY:
                ## move robot command
                pass
            else:
                ppline.play_schedule(snode_schedule)
            break
    else:
        print('homing motion')

None -> ('floor_ws', 0)
('floor_ws', 0) -> ('hold0', 0)
optimal replanning fail
('floor_ws', 0)->('hold0', 0)
hold motion
('hold0', 0) -> ('floor_ws', 0)
('hold0', 0)->('floor_ws', 0)
release motion
('floor_ws', 0) -> ('floor_ws', 1)
optimal replanning fail
('floor_ws', 0)->('floor_ws', 1)
wipe section 1
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(1, 0) = success
branching: 0->1 (0.06/20.0 s, steps/err: 45(56.8768978119 ms)/0.00164179023771)
try: 1 - (1, 0)->(2, 0)
try constrained motion
joint max
constrained motion tried: False
Motion Plan Failure
result: 1 - (1, 0)->(2, 0) = fail
try: 1 - (1, 0)->(2, 0)
try constrained motion
joint max
constrained motion tried: False
Motion Plan Failure
result: 1 - (1, 0)->(2, 0) = fail
try: 1 - (1, 0)->(2, 0)
try constrained motion
joint max
constrained motion tried: False
Motion Plan Failure
result: 1 - (1, 0)->(2, 0) = fail
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: T

## Return to home

In [24]:
snode_last_mobile = snode_schedule_mobile[-1]

In [25]:
ppline.search(snode_last_mobile.state, [('floor_ws', len(wp_list))], verbose=True,
              timeout=0.3, timeout_loop=3, multiprocess=False,
              add_homing=False, terminate_on_first=False, post_optimize=False)
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(snode_schedule)

try: 0 - ('floor_ws', 4)->('floor_ws', 5)
try transition motion
transition motion tried: True
Goal reached
result: 0 - ('floor_ws', 4)->('floor_ws', 5) = success
branching: 0->1 (0.15/3.0 s, steps/err: 42(144.804954529 ms)/0.00132745756318)
try: 0 - ('floor_ws', 4)->('hold0', 4)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 4)->('hold0', 4) = fail
try: 0 - ('floor_ws', 4)->('hold0', 4)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 4)->('hold0', 4) = fail
try: 0 - ('floor_ws', 4)->('hold0', 4)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 4)->('hold0', 4) = fail
try: 0 - ('floor_ws', 4)->('hold0', 4)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 4)->('hold0', 4) = fail
try: 0 - ('floor_ws', 4)->('hold0', 4)
try transition motion
transition motion tried: True
result: 0 - ('floor_ws', 4)->('hold0', 4) = success
branching: 0->2 (0.27/3.0 s, steps/err: 43(86.3659381866 ms)/0.00196201003707)
try: 2 - ('hold0', 4)->('floor_ws', 4)
Motion

reserve 13 -> 10
reserve 13 -> 12
result: 3 - ('floor_ws', 4)->('hold0', 4) = success
branching: 3->13 (2.92/3.0 s, steps/err: 5(95.2911376953 ms)/0.00185648427661)
try: 13 - ('hold0', 4)->('floor_ws', 4)
try transition motion
transition motion tried: True
result: 13 - ('hold0', 4)->('floor_ws', 4) = success
branching: 13->14 (3.04/3.0 s, steps/err: 73(115.48614502 ms)/0.00131970214709)
======================= terminated 0: max iteration time reached (1630697633/1630697630.57 s) ===============================
('floor_ws', 4)->('floor_ws', 5)


## Record initial pose

In [ ]:
xyzw_home = get_xyzw_cur()

# Main Loop

In [18]:
# for i_cn in range(4):
#     cn_cur = CornerSequence[i_cn]
#     if cn_cur == Corners.Right and divisions[0]<2:
#         continue
i_cn = 0

In [ ]:
cn_cur = CornerSequence[i_cn]
print("== Current workspace section: {}".format(cn_cur.name))
section_size = np.concatenate([np.divide(table_prev.dims[:2],  divisions), [TRACK_THICKNESS]])
off_corner = np.divide(table_prev.dims, 2)*corner_point_dirs[cn_cur]
off_sect = (np.divide(table_prev.dims, 2) - np.divide(section_size, 2))*corner_point_dirs[cn_cur]
T_rot_table = SE3(RotationSequence[i_cn], (0,0,0))
T_tf = SE3(np.identity(3), off_sect)
T_ft = SE3_inv(T_tf)

## Add table on relative target location

In [ ]:
track, track_face = add_track(table_prev, TABLE_HEIGHT, area_depth, area_width, corner_center)
T_bt, T_bb2 = calc_base_target(table_prev, track, T_ft, T_rot_table)
gscene.add_highlight_axis("target", "base", "base_link", T_bb2[:3,3], T_bb2[:3,:3])
T_mm2, T_bm2 = base_offet_to_mobile(T_bb2, CONNECT_MOBILE)
gscene.add_highlight_axis("target", "mobile", "base_link", T_bm2[:3,3], T_bm2[:3,:3])

## move mobile robot

In [18]:
cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(T_mm2, CONNECT_MOBILE)
move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw_rd, tar_xyzw, MOBILE_IP, CONNECT_MOBILE,
                  move_direct=cn_cur == Corners.Right)


NameError: name 'T_mm2' is not defined

## update table location

In [8]:
## add moved table in front of robot
table_front = add_table(gscene, "table_front", TABLE_DIMS, T_bt, collision=True)
gscene.NAME_DICT["table"].color = (0.8,0.8,0.8,0.0)
gscene.update_markers_all()

# corner_center_rel = np.abs(np.subtract(tuple(corner_center)+(TABLE_HEIGHT,), table_front.center))

NameError: name 'TABLE_DIMS' is not defined

### Gaze and Refine

In [9]:
## Move to gaze pose
gaze_traj, success = calc_gaze_pose(cn_cur, mplan, table_front, viewpoint, indy, CONNECT_INDY, GAZE_DIST=0.5)
if success:
    gaze_pose = gaze_traj[-1]
    if SHOW_MOTION_RVIZ:
        gscene.show_motion(gaze_traj)

if CONNECT_INDY:
    indy.move_joint_wp(gaze_traj, None, None, None)

# ## return from gaze pose
# if CONNECT_INDY:
#     indy.move_joint_wp(np.array(list(reversed(gaze_traj))), None, None, None)

if CONNECT_INDY:
    with indy:
        Qcur = indy.get_qcur()
else:
    try:
        Qcur = np.load(DATASET_CAM_DIR + '/tablepose_11.npy')
    except Exception as e:
        print(e)

## Refine plane
T_bo_bak = table_front.Toff
table_front = refine_plane(gscene, track, viewpoint, T_ft, Qcur, TABLE_DIMS,
                           True, cn_cur, CAM_HOST, CONNECT_CAM, CONNECT_INDY, ENABLE_DETECT)
table_prev = table_front
T_bo = table_front.Toff


NameError: name 'cn_cur' is not defined

## adjust track

In [19]:
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", section_size,
                           center = T_tf[:3,3], #corner_center_rel*corner_point_dirs[cn_cur],
                           rpy= Rot2rpy(T_tf[:3,:3]), #Rot2rpy(corner_orientations[cn_cur]),
                           color=(0.0,0.8,0.8,0.2), display=True, fixed=True, collision=True,
                           parent="table_front")
track_face = gscene.copy_from(track, new_name="track_face", collision=False, color=(0.8,0.8,0.8,0.0))
TRACK_DIM = np.copy(track_face.dims)
track_face.dims = (3, 3, track.dims[2])
gscene.update_markers_all()

width_range_fit = (np.mean(width_range)-section_size[1]/2, np.mean(width_range)+section_size[1]/2)
sweep_list, track_list = make_work_plane(pscene, track, TOOL_DIM, fix_orientation_front=True)


# init planning

In [20]:
from pkg.planning.constraint.constraint_common             import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
# initial_state = pscene.initialize_state(crob.home_pose)
initial_state = pscene.initialize_state(VIEW_POSE)
print(initial_state.node)

# # remove place points except for the current one
# use_current_place_point_only(pscene, initial_state)

(0, 0)


## search

In [21]:
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()
tplan.prepare()
mplan.update_gscene()

print(initial_state.node)

obj_num = 0
sweep_num = len(sweep_list)
from_state = initial_state.copy(pscene)
from_state.Q = np.array([0]*6)
# from_state.Q = np.array([ 0.        , -0.48869219,  1.48352986, -0.        ,  0.99483767,
#        3.14159265])
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
#     gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01,
                  timeout_loop=20, multiprocess=False, timeout=0.5, timeout_constrained=2,
                  add_homing=False, post_optimize=False)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules(False)
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    if sweep_idx == 0:
        snode_start = snode_schedule[1]
        pscene.set_object_state(snode_schedule[0].state)
        trajectory, success = mplan.planner.plan_joint_motion_py(
            ROBOT_NAME, tuple(snode_start.traj[-1]), tuple(gaze_pose), timeout=1)
        if success:
            snode_start.traj = trajectory
    if sweep_idx == sweep_num-1:
        added_list = ppline.add_return_motion(snode_schedule[-1], initial_state=initial_state, timeout=0.5, try_count=2)
        snode_schedule += added_list
    snode_schedule_ori = snode_schedule
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
#     snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 5, 1)
#     double_sweep_motions(snode_schedule_safe)
#     snode_schedule = snode_schedule_safe
#     snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    snode_schedule = snode_schedule_simple
    from_state = snode_schedule[-1].state
    snode_schedule_all.append(snode_schedule)

(0, 0)
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(1, 0) = success
branching: 0->1 (0.08/20.0 s, steps/err: 44(81.7880630493 ms)/0.00164911427027)
try: 1 - (1, 0)->(2, 0)
try constrained motion
joint min
constrained motion tried: False
Motion Plan Failure
result: 1 - (1, 0)->(2, 0) = fail
try: 1 - (1, 0)->(2, 0)
try constrained motion
joint min
constrained motion tried: False
Motion Plan Failure
result: 1 - (1, 0)->(2, 0) = fail
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(1, 0) = success
branching: 0->2 (0.7/20.0 s, steps/err: 62(76.0741233826 ms)/0.0018968788724)
try: 2 - (1, 0)->(2, 0)
try constrained motion
end
constrained motion tried: True
result: 2 - (1, 0)->(2, 0) = success
branching: 2->3 (1.16/20.0 s, steps/err: 83(451.002120972 ms)/0.0657102313225)
======================= terminated 0: first answer acquired ===============================
try: 0 - (2, 0)->(2, 1)
try trans

## Refine sweep motion

In [22]:
from demo_utils.refine_sweep import simplify_sweep
for snode_schedule in snode_schedule_all:
    simplify_sweep(pscene, mplan, snode_schedule, len_traj=20)

## Play plan

In [23]:
if SHOW_MOTION_RVIZ:
    for snode_schedule in snode_schedule_all:
        ppline.play_schedule(snode_schedule, period=0.1)

## Execute plan

In [25]:
def fn_move_indy():
    print("execute one task")
    if CONNECT_INDY:
        for snode_schedule in snode_schedule_all:
            ppline.execute_schedule(snode_schedule, one_by_one=True)
            with indy:
                time.sleep(0.5)
                indy.wait_for_move_finish()

        with indy:
            indy.joint_move_to(np.rad2deg(VIEW_POSE))
            time.sleep(0.5)
            indy.wait_for_move_finish()


In [32]:
fn_move_indy()
if cn_cur == Corners.Left and divisions[0]>=2:
    cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(SE3(np.identity(3), [0,section_size[1], 0]), CONNECT_MOBILE)
    ref_xyzw = cur_xyzw
    delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
    for i_hori in range(1,divisions[0]-1):
        tar_xyzw = delta_xyzw*i_hori + ref_xyzw
        cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
        fn_move_indy()

execute one task
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move finished!
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move finished!
binder: brush_face
rname: indy0
binder: None
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move finished!
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move finished!
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move finished!
binder: brush_face
rname: indy0
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Connect: Server IP (192.168.0.3)
Move finished!
binder: brush_f

In [30]:
indy.traj_vel = 3

# Go home after loop

In [ ]:
# cur_xyzw = get_xyzw_cur()
# cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, xyzw_home, xyzw_home, MOBILE_IP, CONNECT_MOBILE, move_direct=False)

## Re-adjust location

In [26]:
# T_bb2 = np.matmul(T_bo, SE3_inv(T_bo_bak))
# gscene.add_highlight_axis("target", "base", "base_link", T_bb2[:3,3], T_bb2[:3,:3])

# T_mm2, T_bm2 = base_offet_to_mobile(T_bb2, CONNECT_MOBILE)

# cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(T_mm2, CONNECT_MOBILE)
# cur_xyzw = move_mobile_robot(cur_xyzw, tar_xyzw_rd, tar_xyzw, CONNECT_MOBILE)

# T_bo = T_bo_bak
# table_front = gscene.create_safe(gtype=GEOTYPE.BOX, name="table_front", link_name="base_link",
#                                  dims=TABLE_DIMS, center=T_bo[:3, 3], rpy=Rot2rpy(T_bo[:3, :3]),
#                                  color=(0.8, 0.8, 0.8, 0.5), display=True, fixed=True, collision=False)

## Test sweep manually

In [ ]:
# from pkg.utils.rotation_utils import *
# from pkg.planning.constraint.constraint_subject import SweepLineTask
# from pkg.utils.utils import list2dict

# def get_jacobian(gscene, gtem, Q):
#     Q_dict = list2dict(Q, gscene.joint_names)
#     Jac = []
#     for ij, jname in enumerate(gscene.joint_names):
#         joint = gscene.urdf_content.joint_map[jname]
#         Tj = T_xyzrpy((joint.origin.xyz, joint.origin.rpy))
#         T_link = get_tf(joint.parent, Q_dict, gscene.urdf_content)
#         T_bj = np.matmul(T_link, Tj)
#         zi = np.matmul(T_bj[:3, :3], joint.axis)
#         T_p = gtem.get_tf(Q_dict)
#         dpi = T_p[:3, 3] - T_bj[:3, 3]
#         zp = np.cross(zi, dpi)
#         Ji = np.concatenate([zp, zi])
#         Jac.append(Ji)
#     Jac = np.array(Jac).transpose()
#     return Jac

# for k, v in ppline.tplan.snode_dict.items():
#     gscene.show_motion(snode.traj)


#     SINGULARITY_CUT = 0.01
#     len_traj = 100

#     Qi = snode.traj[-1]
#     Qidict = list2dict(Qi, gscene.joint_names)
#     Ti = gtem.get_tf(Qidict)

#     dP = wp2.Toff[:3,3] - wp1.Toff[:3,3]
#     dPabs = np.linalg.norm(dP)
#     DP = dPabs / len_traj
#     DIR = np.concatenate([dP / dPabs, [0] * 3])
#     Q = Qi


#     singularity = False
#     Traj_wipe = [Qi]

#     for _ in range(len_traj):
#         Jac = get_jacobian(gscene, gtem, Q)
#         if np.min(np.abs(np.real(np.linalg.svd(Jac)[1]))) <= SINGULARITY_CUT:
#             singularity = True
#             print("singular")
# #             break
#         Jinv = np.linalg.inv(Jac)
#         dQ = np.matmul(Jinv, np.multiply(DIR, DP))
#         Q = Q + dQ
#         Traj_wipe.append(Q)
#         dlim = np.subtract(RobotSpecs.get_joint_limits(RobotType.indy7), Q[:, np.newaxis])
#         if np.any(dlim[:, 0] > 0):
#             print("min lim: {}".format(np.where(dlim[:, 0] > 0)[0]))
#             break
#         if np.any(dlim[:, 1] < 0):
#             print("max lim: {}".format(np.where(dlim[:, 1] < 0)[0]))
#             break
#     #         if not mplan.validate_trajectory([Q]):
#     #             print("col")
#     #             break
#     #         Tnew = gtem.get_tf(list2dict(Q, gscene.joint_names))
#     #         if np.abs(Ti[0,3]-Tnew[0,3])>0.01:
#     #             print("off")
#     #             break
#     gscene.show_motion(Traj_wipe)

#     Traj_wipe = np.array(Traj_wipe)

## test saved param

In [ ]:
# Tcur = get_tf(mplan.sweep_params[0], mplan.sweep_params[1], mplan.sweep_params[2], mplan.sweep_params[3])
# T_tar_tool = mplan.sweep_params[-1]

# ee_point = gscene.create_safe(GEOTYPE.SPHERE, "end_point", TIP_LINK, (0.01,)*3, 
#                               center=(0,0,0), rpy=(0,0,0), 
#                               color=(0.8,0.2,0.2,0.8), display=True, fixed=True, collision=False)

# from_Q_dict = mplan.sweep_params[1]
# from_Q = dict2list(from_Q_dict, gscene.joint_names)

# trajectory = get_sweep_traj(mplan, brush_face.geometry, np.subtract(T_tar_tool[:3,3], Tcur[:3, 3]),
#                             from_Q, DP=0.01, ERROR_CUT=0.01, SINGULARITY_CUT = 0.01, VISUALIZE=True, 
#                             VERBOSE=True)

# Tnew = get_tf(mplan.sweep_params[0], list2dict(trajectory[-1], gscene.joint_names), mplan.sweep_params[2], mplan.sweep_params[3])
# success = np.sum(np.abs(Tcur[:3,3]-Tnew[:3,3]))<1e-2